# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
# store the CSV in WeatherPy into a DataFrame

city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,78.53,65,40,10.36,US,1638132336
1,1,cidreira,-30.1811,-50.2056,72.30,87,100,8.61,BR,1638132338
2,2,punta arenas,-53.1500,-70.9167,50.11,62,75,29.93,CL,1638132158
3,3,ostersund,63.1792,14.6357,18.05,93,99,6.91,SE,1638132345
4,4,ola,59.5833,151.2833,14.83,87,100,4.21,RU,1638132347


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
#configure gmaps

gmaps.configure(api_key=g_key)

In [23]:
# heatmap of the humidity

locations = city_data_df [["Lat","Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure()

#add the heatmap layer to the figure

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] <80) & \
    (city_data_df["Wind Speed"] <10) & \
    (city_data_df["Cloudiness"] == 0)
].dropna()




idealWeatherDF.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,salalah,17.0151,54.0924,75.29,73,0,4.61,OM,1638132493
68,68,guerrero negro,27.9769,-114.0611,78.13,34,0,8.57,MX,1638132375
91,91,lahij,13.1667,44.5833,73.98,63,0,1.90,YE,1638132583
189,189,santa rosalia,27.3167,-112.2833,72.88,48,0,9.66,MX,1638132840
263,263,jati,24.3539,68.2671,73.76,28,0,8.52,PK,1638133029


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hoteldf = idealWeatherDF[["City", "Country", "Lat", "Lng"]]
hoteldf["Hotel Name"] = ""
hoteldf

<ipython-input-25-b92fa79e55e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hoteldf["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
61,salalah,OM,17.0151,54.0924,
68,guerrero negro,MX,27.9769,-114.0611,
91,lahij,YE,13.1667,44.5833,
189,santa rosalia,MX,27.3167,-112.2833,
263,jati,PK,24.3539,68.2671,
264,aswan,EG,24.0934,32.9070,
288,bermejo,AR,-32.8833,-68.8000,
299,muscat,OM,23.6139,58.5922,
326,morondava,MG,-20.2833,44.2833,
361,kanye,BW,-24.9667,25.3327,


In [30]:
#set parameters to search hotels

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
    
}

#Loop  through the dataframe using iterrows()

for index, row in hoteldf.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    #use the latitude and long to set the location value in the params dic.
    
    params["location"] = f"{lat},{lng}"
    
    #use google api places base url to search 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # get requests and pring the url 
    
    name_address = requests.get(base_url, params=params).json()
    
#     pprint(name_address) # print ["results"][0]["name"]

    
    try:
        
        hoteldf.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except:
        
        print (f"Error finding hotel in {row['City']}, Bypass")
        pass
        

hoteldf
       

Error finding hotel in lahij, Bypass


,City,Country,Lat,Lng,Hotel Name
61,salalah,OM,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
68,guerrero negro,MX,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
91,lahij,YE,13.1667,44.5833,
189,santa rosalia,MX,27.3167,-112.2833,Hotel Las Casitas Santa Rosalia
263,jati,PK,24.3539,68.2671,Cafe Lala Rehan Restaurant
264,aswan,EG,24.0934,32.9070,Nuba Nile
288,bermejo,AR,-32.8833,-68.8000,Muras Apart Hotel
299,muscat,OM,23.6139,58.5922,Mutrah Hotel
326,morondava,MG,-20.2833,44.2833,La Case Bambou
361,kanye,BW,-24.9667,25.3327,The Savanna Lodge


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
locations = hoteldf[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)


# Display figure

fig

Figure(layout=FigureLayout(height='420px'))